<a href="https://colab.research.google.com/github/mafer09/craving-meter/blob/main/get_brandId_productId.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#imports
import json
import re

Two methods to authenticate against google drive folder `COMP631-CravingMeter/Files` (location on which the api generated files are hosted)

In [2]:
#mount to google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
#PyDrive google authentication set up
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
folder_path = '/content/gdrive/MyDrive/COMP631-CravingMeter/'

Retrieve products file metadata from google drive folder and write `file_id` and `file_name` into `files-folder-contents.txt` file

In [6]:
# Retrieves googleDrive metadata of files contained within 'COMP631-CravingMeter/Files" folder
# CODE TAKES A WHILE TO RUN
folder_id = '1SFId5EqIW_iaCOhB1tLlp-7shHtI1xoB' # folder_id for 'COMP631-CravingMeter/Files'
file_list = drive.ListFile({'q': f"'{folder_id}' in parents and trashed=false"}).GetList()

In [ ]:
#Write file folder contents (title and id) into .txt file
file_name = 'files-folder-contents.txt'

with open(folder_path+file_name, 'w') as writefile:
  for f_i, file in enumerate(file_list):
    f_id = file_list[f_i]['id']
    f_title = file_list[f_i]['title']
    line = f_id+","+f_title+"\n"
    writefile.write(line)

Read in `files-folder-contents.txt`, open each file, and extract `brand_name`, `item_name`, `item_id` and write it into `item_fields.txt` file 

In [ ]:
#Read in list of files **requires google mount
file_name = 'files-folder-contents.txt'
with open(folder_path+file_name) as f:
  item_list = f.readlines()

In [ ]:
#get index of last file retrieved
last_place= "5e27fd7ec3e834760d32f414.json\n"
for idx, item in enumerate(item_list):
  if item.endswith(last_place):
    last_index = idx
# [y[1] for y in item_list].index(last_place)
last_index += 1
print(last_index)

83643


In [ ]:
#Write brand_name, item_name and item_id into .txt file
file_name = 'item_fields.txt'

with open(folder_path+file_name, 'a') as writefile:
  # url_fields_list = []
  for item in item_list[last_index:-1]:
    try:
      res = tuple(map(str,item.split(',')))
      file_id = res[0]
      get_file = drive.CreateFile({'id': file_id})
      file_content = get_file.GetContentString()
      file_json = json.loads(file_content)
      
      brand_name = file_json['fields']['brand_name']
      item_name = file_json['fields']['item_name']
      item_id = file_json['fields']['item_id']
      # url_fields_list.append((brand_name, item_name, item_id))
      line = "\""+brand_name+"\",\""+item_name+"\","+item_id+"\n"
      writefile.write(line)
    except FileNotFoundError: 
      print("error with google drive")
